In [2]:
from elasticsearch import Elasticsearch

In [3]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","pDjStGnelbMpjssD9QqR"),
    ca_certs = "/Users/someshs/Desktop/internship workspace 2024/elasticsearch-8.15.2/config/certs/http_ca.crt"

)
es.ping()

True

## prepare the data

In [4]:
import pandas as pd

df = pd.read_csv("myntra_products_catalog.csv").loc[:499]
df.head()


,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


## convert the relevant field to a vector using BERT model


In [5]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           468
                                                                                   True             32
Name: count, dtype: int64

In [6]:
df.fillna("None",inplace = True)

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")


/Users/someshs/Desktop/internship workspace 2024/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/someshs/Desktop/internship workspace 2024/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))

In [9]:
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645748, -0.0026341842, -0.0035884231, 0...."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.024660734, -0.028755423, -0.020332452, 0.0..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.046943296, 0.08182791, 0.048335195, -0.000..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.0150987785, -0.0102855, 0.00948732, -0.023..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[-0.017746517, 0.0062095528, 0.02181398, 0.026..."


In [15]:
es.ping()

True

## create new index in elastic search!

In [18]:
from indexMapping import indexMapping
es.indices.create(index = "all_products",mappings = indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

## injest the data into index

In [19]:
record_list = df.to_dict("records")

In [20]:
record_list[0]

{'ProductID': 10017413,
 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag',
 'ProductBrand': 'DKNY',
 'Gender': 'Unisex',
 'Price (INR)': 11745,
 'NumImages': 7,
 'Description': 'Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer',
 'PrimaryColor': ' Black',
 'DescriptionVector': array([ 2.76457481e-02, -2.63418420e-03, -3.58842313e-03,  5.13588227e-02,
         3.09660006e-02,  1.40507007e-02,  7.27052568e-03,  3.13871726e-02,
        -6.23787753e-02, -3.82877584e-03,  3.15214507e-02,  7.55473226e-02,
         2.12644227e-03,  4.64894176e-02,  5.07448912e-02, -1.71941910e-02,
         1.22891720e-02, -1.95682552e-02, -9.6

In [21]:
for record in record_list:
    try:
        es.index(index = "all_products",document = record,id = record["ProductID"])
    except Exception as e:
        print(e)

In [22]:
es.count(index = "all_products")

ObjectApiResponse({'count': 500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## search the data

In [24]:
input_keyword = "Blue_shoes"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates":500,

}
res = es.knn_search(index = "all_products",knn = query,source = ["ProductName","Description"])
res["hits"]["hits"]

/var/folders/q3/k2msdb912j7_3txnb0w6bx_c0000gn/T/ipykernel_3563/2566722302.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index = "all_products",knn = query,source = ["ProductName","Description"])


[{'_index': 'all_products',
  '_id': '10018013',
  '_score': 0.52967405,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}},
 {'_index': 'all_products',
  '_id': '10018075',
  '_score': 0.52967405,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}}]